In [ ]:
import numpy as np
import multiprocessing as mp
import random,copy,string
from nltk.tokenize import word_tokenize
from scipy.stats import pearsonr
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Convolution1D, MaxPooling1D, Flatten
from tensorflow.python.keras.layers import Lambda, multiply, concatenate, Dense
from tensorflow.python.keras.regularizers import l2
from tensorflow.python.keras.callbacks import Callback
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


In [ ]:
import os, string, random, time, math
import torch.optim as optim
import matplotlib.pyplot as plt
from IPython.display import clear_output
import torch
'''import mlflow
import mlflow.pytorch'''

'import mlflow\nimport mlflow.pytorch'

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
#cd 'drive/My Drive'

/content/drive/My Drive


In [ ]:
#ls

 2017_takelab+cnn/                     sts-dev.csv
 2017_takelab_MLP/                     sts-test.csv
 2017_takelab_SVR/                     sts-train.csv
 cc.en.300.bin                         takelab_jc.txt
'Colab Notebooks'/                     takelab_lin.txt
 fasttext_cross_cnn_code.ipynb         takelab_output_MSRpar.txt
 full.txt                              temp.ipynb
'Getting started.pdf'                  temp_word2vec_kl.ipynb
 GoogleNews-vectors-negative300.bin    thesis/
 jc_lin.txt                            word2vec_cross_adam_model.pt
 model_cifar.pt                        word2vec_cross_cnn_code.ipynb
 processed_full.txt                    word2vec_cross_epoch.pt
 processed_jc_lin.txt                  word2vec_kl_cnn_code.ipynb
 processed_takelab_jc.txt              word2vec_kl_epoch.pt
 processed_takelab_lin.txt             word2vec_kl_model.pt
 processed_takelab_output_MSRpar.txt   wv_wlsa.txt
 processed_wv_wlsa.txt                 wv_wwv.txt
 processed_wv_wwv.txt


In [ ]:
c = dict()
c['num_runs']   = 3
c['num_epochs'] = 2
c['num_batchs'] = 4
c['batch_size'] = 3
c['wordvectdim']  = 300
c['sentencepad']  = 60
c['num_classes']  = 6
c['cnnfilters']     = {1: 1800}
c['cnninitial']     = 'he_uniform'
c['cnnactivate']    = 'relu'
c['densedimension'] = list([1800])
c['denseinitial']   = 'he_uniform'
c['denseactivate']  = 'tanh'
c['optimizer']  = 'adam'

In [ ]:
wordvectdim=300
wordtoindex= dict()
indextovector= []
indextovector.append(np.zeros(wordvectdim))    

# Loading word vectors

In [ ]:
!pip3 install fasttext

     |████████████████████████████████| 71kB 2.4MB/s 
  Created wheel for fasttext: filename=fasttext-0.9.2-cp36-cp36m-linux_x86_64.whl size=3018054 sha256=3042350d7f540e2b278c7ce0c31c2ef9025b9b956e9fc0164caac1857e7fdf1e
  Stored in directory: /root/.cache/pip/wheels/98/ba/7f/b154944a1cf5a8cee91c154b75231136cc3a3321ab0e30f592
Successfully built fasttext


In [ ]:
import fasttext

In [ ]:
ft = fasttext.load_model('../cc.en.300.bin')

# Params

In [ ]:
class Params(object):
  def __init__(self, batch_size, epochs, seed, log_interval):
    self.batch_size = batch_size
    self.epochs = epochs
    self.seed = seed
    self.log_interval = log_interval

args= Params(16,6,0,8)

# Load Data

In [ ]:
def matrixize(sentencelist, sentencepad):
  padding= np.zeros(300)
  matrix= np.zeros((len(sentencelist),sentencepad,300 ))
  m=0
  s=1

  for i in range(len(sentencelist)):
    for j in range(len(sentencelist[i])):
      try:
        matrix[i][j]= ft.get_word_vector(sentencelist[i][j])
      except:
        print(sentencelist[i][j])
        matrix[i][j]= np.random.normal(m,s,300)/norm(np.random.normal(m,s,300) + 1e-8)

  return matrix


In [ ]:
def _load_data(filename):
        s0,s1,labels = [],[],[]
        lines=open(filename,'r',encoding='utf-8').read().splitlines()
        for line in lines:
            _,_,_,_, label, s0x, s1x = line.rstrip().split('\t')[:7]
            labels.append(float(label))
            s0.append([word.lower() for word in word_tokenize(s0x) if word not in string.punctuation])
            s1.append([word.lower() for word in word_tokenize(s1x) if word not in string.punctuation])

        m0 = matrixize(s0, c['sentencepad'])
        m1 = matrixize(s1, c['sentencepad'])
        classes = np.zeros((len(labels),c['num_classes']))
        for i, label in enumerate(labels): # making probability distribution of classes
            if np.floor(label) + 1 < c['num_classes']:
                classes[i, int(np.floor(label)) + 1] = label - np.floor(label)
            classes[i, int(np.floor(label))] = np.floor(label) - label + 1
            
        return {'labels': labels, 's0': s0, 's1': s1, 'classes': classes, 'm0': m0, 'm1': m1}


# Sampling Batch

In [ ]:
def _sample_pairs(data,batch_size):
  datacopy={}
  for i in data.keys():
    datacopy[i]= []
  for i in range(batch_size):
    index = np.random.randint(len(data['labels']))
    #print(index)
    for key,value in data.items():
      datacopy[key].append(value[index])
  datacopy['classes']= torch.tensor(datacopy['classes'])
  datacopy['m0']= torch.tensor(datacopy['m0'])
  datacopy['m1']= torch.tensor(datacopy['m1'])
  return datacopy

# Defining CNN Model

In [ ]:
import torch.nn as nn

In [ ]:
class CNN_Model(nn.Module):
  def __init__(self,nh):
    super(CNN_Model, self).__init__()
    self.cnn= nn.Sequential(nn.Conv1d(300,1800,1),              #(bs,300,60) -> (bs, 1800, 60)
                          
                            nn.LeakyReLU(0.01),
                            nn.MaxPool1d(kernel_size=60))       #(bs,1800,60)-> (bs, 1800, 1)

    self.Linear= nn.Sequential(nn.Linear(3600,nh),            #(bs, 3600)  -> (bs, 1800 )
                               nn.LeakyReLU(0.01),
                               nn.Linear(nh,6),               #(bs,1800)   -> (bs,6)
                               nn.Softmax(dim=1))
                            
    

  def forward(self,input1, input2):
    input1= self.cnn(input1)
    input2= self.cnn(input2)
    #print("input1=",input1.shape)
    #print("input2=",input2.shape)
    input1= torch.flatten(input1,1)
    input2= torch.flatten(input2,1)
    #print("input1=",input1.shape)
    #print("input2=",input2.shape)
    absdiff= abs(input1 - input2)
    mulDifference= input1 * input2
    concatenate = torch.cat((absdiff,mulDifference),1)
    #print(concatenate.shape)
    output= self.Linear(concatenate)
    #print(output.shape)
    return (output)

In [ ]:
def weights_init(m):
    if isinstance(m, nn.Conv1d) or isinstance(m, torch.nn.Linear):
        torch.nn.init.kaiming_normal_(m.weight)
        torch.nn.init.zeros_(m.bias)


In [ ]:
'''class CNN_Model(nn.Module):
  def __init__(self,nh=1800):
    super(CNN_Model, self).__init__()
    self.bi_cnn= nn.Sequential(nn.Conv1d(300,600,2),                   #(bs,300,60) -> (bs,600, 59)
                                nn.ReLU(0.01),
                                nn.MaxPool1d(kernel_size=59))           #(bs,600,59) -> (bs,600, 1) 


    self.tri_cnn= nn.Sequential(nn.Conv1d(300,600,3),                    #(bs,300,60) -> (bs,600, 58)
                                nn.ReLU(0.01),
                                nn.Dropout(0.4),
                                nn.MaxPool1d(kernel_size=58))             #(bs,300,60) -> (bs,600,1)

    self.four_cnn= nn.Sequential(nn.Conv1d(300,600,4),                   #(bs,300,60) -> (bs,600, 57)
                                nn.ReLU(0.01),
                                nn.Dropout(0.4),
                                nn.MaxPool1d(kernel_size=57))           #(bs,600,59) -> (bs,600, 1)                                       


    self.Linear= nn.Sequential(nn.Linear(3600,nh),            #(bs, 3600)  -> (bs, 1800 )
                               nn.ReLU(0.01),
                               nn.Linear(nh,6),               #(bs,1800)   -> (bs,6)
                               nn.Softmax(dim=1))
                            
    

  def forward(self,input1, input2):
    inp1_1= self.bi_cnn(input1)
    inp1_2= self.tri_cnn(input1)
    inp1_3= self.four_cnn(input1)
    concatenate1 = torch.cat((inp1_1,inp1_2,inp1_3),1)

    inp2_1= self.bi_cnn(input2)
    inp2_2= self.tri_cnn(input2)
    inp3_3= self.four_cnn(input2)
    concatenate2 = torch.cat((inp1_1,inp1_2,inp1_3),1)

    input1= torch.flatten(concatenate1,1)
    input2= torch.flatten(concatenate2,1)

    absdiff= abs(input1 - input2)
    mulDifference= input1 * input2

    concatenate = torch.cat((absdiff,mulDifference),1)
    output= self.Linear(concatenate)

    return (output)'''

'class CNN_Model(nn.Module):\n  def __init__(self,nh=1800):\n    super(CNN_Model, self).__init__()\n    self.bi_cnn= nn.Sequential(nn.Conv1d(300,600,2),                   #(bs,300,60) -> (bs,600, 59)\n                                nn.ReLU(0.01),\n                                nn.MaxPool1d(kernel_size=59))           #(bs,600,59) -> (bs,600, 1) \n\n\n    self.tri_cnn= nn.Sequential(nn.Conv1d(300,600,3),                    #(bs,300,60) -> (bs,600, 58)\n                                nn.ReLU(0.01),\n                                nn.Dropout(0.4),\n                                nn.MaxPool1d(kernel_size=58))             #(bs,300,60) -> (bs,600,1)\n\n    self.four_cnn= nn.Sequential(nn.Conv1d(300,600,4),                   #(bs,300,60) -> (bs,600, 57)\n                                nn.ReLU(0.01),\n                                nn.Dropout(0.4),\n                                nn.MaxPool1d(kernel_size=57))           #(bs,600,59) -> (bs,600, 1)                                       \

# Defining Loss

In [ ]:
def _lossfunction(y_true,y_pred):
    
    #print("y_true.shape",y_true.shape)
    #print("y_pred.shape",y_pred.shape)
    ny_true = y_true[:,1] + 2*y_true[:,2] + 3*y_true[:,3] + 4*y_true[:,4] + 5*y_true[:,5]
    #print(ny_true)
    ny_pred = y_pred[:,1] + 2*y_pred[:,2] + 3*y_pred[:,3] + 4*y_pred[:,4] + 5*y_pred[:,5]
    #print(ny_pred)
    my_true = (ny_true.mean())
    #print(my_true)
    my_pred = (ny_pred.mean())
    #print(my_pred)
    var_true = (ny_true - my_true)**2
    var_pred = (ny_pred - my_pred)**2
    eps= 1e-14
    return -torch.sum((ny_true-my_true)*(ny_pred-my_pred),axis=-1) / (torch.sqrt(torch.sum(var_true,axis=-1)* torch.sum(var_pred,axis=-1)+eps))

# Training

## Train setup

In [ ]:
def train_batch(net,trainload, opt,batch_size, device= 'cpu'):
  
  net.train().to(device)
  opt.zero_grad()
  
  

  data= _sample_pairs(trainload,batch_size)
  input1= (torch.transpose(data['m0'],1,2)).to(device,dtype= torch.float32)
  input2= (torch.transpose(data['m1'],1,2)).to(device,dtype= torch.float32)
  true_output= (data['classes']).to(device)
  pred_output= net(input1,input2)
  loss= _lossfunction(true_output,pred_output)

  #loss= criterion(pred_output,true_output)
  loss.backward()
  opt.step()
  #print("lossgf",loss.item())
  return loss

## Validation Setup

In [ ]:
def valid_batch(net,validation, opt,batch_size, device= 'cpu'):
  net.eval().to(device)
  
  data = _sample_pairs(validation,batch_size)
  input1= (torch.transpose(data['m0'],1,2)).to(device,dtype= torch.float32)
  input2= (torch.transpose(data['m1'],1,2)).to(device,dtype= torch.float32)
  true_output= (data['classes']).to(device)
  pred_output= net(input1,input2)
  loss= _lossfunction(true_output,pred_output)
  #loss= criterion(pred_output,true_output)
  return loss


# Full Training Setup

## Adam

In [ ]:
def train_setup_Adam(net,lr = 0.0001,  batch_size = args.batch_size, momentum = 0.9, device = 'cpu',epoch= args.epochs):
  torch.autograd.set_detect_anomaly(True)
  valid_loss_min = np.Inf
  net.apply(weights_init)
  opt = optim.Adam(net.parameters(), lr=lr)
  trainload= _load_data('sts-train.csv')
  validation= _load_data('sts-dev.csv')
  train_n_batch= int(len(trainload['labels'])/batch_size)
  train_loss_arr = np.zeros(train_n_batch+ 1)
  ###################
  # train the model #
  ###################
  
  for epoch in range(epoch):
    if epoch != 0:
      net.load_state_dict(torch.load('fasttext_cross_epoch.pt'))
    for i in range(train_n_batch):
      train_loss_arr[i+1] = (train_loss_arr[i]*i + train_batch(net,trainload, opt, batch_size, device))/(i + 1)
      #print("train_loss_MSE",train_loss_arr[i+1])

     
   
    
      #if i%display_freq == display_freq-1:
      '''clear_output(wait=True)
      print('Iteration', i, 'Loss', loss_arr[i])
      plt.figure()
      plt.plot(train_loss_arr,'-*')
      plt.xlabel('Iteration')
      plt.ylabel('Loss')
      plt.show()
      print('\n\n')'''

      ######################    
      # validate the model #
      ######################
    valid_n_batch = int(len(validation['labels'])/batch_size)
    valid_loss_arr = np.zeros(valid_n_batch + 1)
    for i in range(valid_n_batch):
      valid_loss_arr[i+1] = (valid_loss_arr[i]*i + valid_batch(net,validation, opt, batch_size, device))/(i + 1)
    
    '''mlflow.log_metric('train_loss_MSE', train_loss_arr[i+1])
    mlflow.log_metric('valid_loss_MSE', valid_loss_arr[i+1])'''

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss_arr[train_n_batch ], valid_loss_arr[valid_n_batch ]))
  
    if valid_loss_arr[valid_n_batch ] <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min, valid_loss_arr[valid_n_batch ]))
          torch.save(net.state_dict(), 'fasttext_cross_epoch.pt')
          valid_loss_min = valid_loss_arr[valid_n_batch ]


In [ ]:
net= CNN_Model(2048)
train_setup_Adam(net, lr= 0.0005, epoch= 18)

Epoch: 0 	Training Loss: -0.630394 	Validation Loss: -0.716440
Validation loss decreased (inf --> -0.716440).  Saving model ...
Epoch: 1 	Training Loss: -0.817180 	Validation Loss: -0.733017
Validation loss decreased (-0.716440 --> -0.733017).  Saving model ...
Epoch: 2 	Training Loss: -0.886970 	Validation Loss: -0.776813
Validation loss decreased (-0.733017 --> -0.776813).  Saving model ...
Epoch: 3 	Training Loss: -0.910880 	Validation Loss: -0.766688
Epoch: 4 	Training Loss: -0.912935 	Validation Loss: -0.785050
Validation loss decreased (-0.776813 --> -0.785050).  Saving model ...
Epoch: 5 	Training Loss: -0.935186 	Validation Loss: -0.786898
Validation loss decreased (-0.785050 --> -0.786898).  Saving model ...
Epoch: 6 	Training Loss: -0.952041 	Validation Loss: -0.793033
Validation loss decreased (-0.786898 --> -0.793033).  Saving model ...
Epoch: 7 	Training Loss: -0.963446 	Validation Loss: -0.796177
Validation loss decreased (-0.793033 --> -0.796177).  Saving model ...
Epoch

## Adamax

In [ ]:
def train_setup_Adamax(net,lr = 0.0001,  batch_size = args.batch_size, momentum = 0.9, device = 'cpu',epoch= args.epochs):
  torch.autograd.set_detect_anomaly(True)
  valid_loss_min = np.Inf
  net.apply(weights_init)
  opt = optim.Adamax(net.parameters(), lr=lr)
  trainload= _load_data('sts-train.csv')
  validation= _load_data('sts-dev.csv')
  train_n_batch= int(len(trainload['labels'])/batch_size)
  train_loss_arr = np.zeros(train_n_batch+ 1)
  ###################
  # train the model #
  ###################
  
  for epoch in range(epoch):
    if epoch != 0:
      net.load_state_dict(torch.load('fasttext_cross_epoch.pt'))
    for i in range(train_n_batch):
      train_loss_arr[i+1] = (train_loss_arr[i]*i + train_batch(net,trainload, opt, batch_size, device))/(i + 1)
      #print("train_loss_MSE",train_loss_arr[i+1])

     
   
    
      #if i%display_freq == display_freq-1:
      '''clear_output(wait=True)
      print('Iteration', i, 'Loss', loss_arr[i])
      plt.figure()
      plt.plot(train_loss_arr,'-*')
      plt.xlabel('Iteration')
      plt.ylabel('Loss')
      plt.show()
      print('\n\n')'''

      ######################    
      # validate the model #
      ######################
    valid_n_batch = int(len(validation['labels'])/batch_size)
    valid_loss_arr = np.zeros(valid_n_batch + 1)
    for i in range(valid_n_batch):
      valid_loss_arr[i+1] = (valid_loss_arr[i]*i + valid_batch(net,validation, opt, batch_size, device))/(i + 1)
    
    '''mlflow.log_metric('train_loss_MSE', train_loss_arr[i+1])
    mlflow.log_metric('valid_loss_MSE', valid_loss_arr[i+1])'''

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(
          epoch, train_loss_arr[train_n_batch ], valid_loss_arr[valid_n_batch ]))
  
    if valid_loss_arr[valid_n_batch ] <= valid_loss_min:
          print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(
          valid_loss_min, valid_loss_arr[valid_n_batch ]))
          torch.save(net.state_dict(), 'fasttext_cross_epoch.pt')
          valid_loss_min = valid_loss_arr[valid_n_batch ]


In [ ]:
'''net= CNN_Model()
train_setup(net,device='cuda:0')'''


# Evaluate

## Evaluation Setup

In [ ]:
def eval_setup(net,batch_size = 1379, device = 'cpu'):
  
  net.load_state_dict(torch.load('fasttext_cross_epoch.pt'))
  net.eval().to(device)
  testload= _load_data('sts-test.csv')
  test_n_batch= int(len(testload['labels'])/batch_size)
  #print(len(testload['labels']))
  test_loss_arr = np.zeros(test_n_batch+ 1)


  data= _sample_pairs(testload,batch_size)
  input1= (torch.transpose(data['m0'],1,2)).to(device,dtype= torch.float32)
  input2= (torch.transpose(data['m1'],1,2)).to(device,dtype= torch.float32)
  pred_output= (net(input1,input2)).to('cpu')
  pred= pred_output.detach().numpy()
  prediction = np.dot(np.array(pred),np.arange(c['num_classes']))
  print('pred_shape',prediction.shape)
  goldlabels = data['labels']
  result=round((pearsonr(prediction, goldlabels)[0]),4)
  print("result",result)

  return result


In [ ]:
net=CNN_Model(2048)
eval_setup(net,device = 'cuda:0')

pred_shape (1379,)
result 0.738


0.738

# Hyper-parameter Tuning

## optim.Adam

In [ ]:
best_model_score = -(np.Inf)
for learning_rate in [0.0005,0.00055]:
    
        for hidden_nodes in [1600,1800,1900]:
        
          expt_id = '%d_%d' % (int(learning_rate*100),  hidden_nodes)
          print('\nLR = %.8f, Hidden nodes = %d\n' % (learning_rate, hidden_nodes))

          model = CNN_Model(hidden_nodes)
          train_setup_Adam(model,lr=learning_rate,device='cuda:0',epoch= args.epochs)

          eval_score=eval_setup(model,device = 'cuda:0')

          if eval_score >= best_model_score :
            print('best_model_score increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            best_model_score, eval_score))
            torch.save(model.state_dict(), 'fasttext_cross_adam_model.pt')
            best_model_score= eval_score

            

          

          




LR = 0.00050000, Hidden nodes = 1600

Epoch: 0 	Training Loss: -0.693432 	Validation Loss: -0.741591
Validation loss decreased (inf --> -0.741591).  Saving model ...
Epoch: 1 	Training Loss: -0.827671 	Validation Loss: -0.767782
Validation loss decreased (-0.741591 --> -0.767782).  Saving model ...
Epoch: 2 	Training Loss: -0.884130 	Validation Loss: -0.789634
Validation loss decreased (-0.767782 --> -0.789634).  Saving model ...
Epoch: 3 	Training Loss: -0.920197 	Validation Loss: -0.799794
Validation loss decreased (-0.789634 --> -0.799794).  Saving model ...
Epoch: 4 	Training Loss: -0.946202 	Validation Loss: -0.785072
Epoch: 5 	Training Loss: -0.943323 	Validation Loss: -0.784964
pred_shape (1379,)
result 0.7396
best_model_score increased (-inf --> 0.739600).  Saving model ...

LR = 0.00050000, Hidden nodes = 1800

Epoch: 0 	Training Loss: -0.667266 	Validation Loss: -0.715252
Validation loss decreased (inf --> -0.715252).  Saving model ...
Epoch: 1 	Training Loss: -0.819648 	Val

## optim.Adamax

In [ ]:
best_model_score = -(np.Inf)
for learning_rate in [0.0005,0.00055]:
    
        for hidden_nodes in [1600,1800,1900]:
        
          expt_id = '%d_%d' % (int(learning_rate*100),  hidden_nodes)
          print('\nLR = %.8f, Hidden nodes = %d\n' % (learning_rate, hidden_nodes))

          model = CNN_Model(hidden_nodes)
          train_setup_Adamax(model,lr=learning_rate,device='cuda:0',epoch= args.epochs)

          eval_score=eval_setup(model,device = 'cuda:0')

          if eval_score >= best_model_score :
            print('best_model_score increased ({:.6f} --> {:.6f}).  Saving model ...'.format(
            best_model_score, eval_score))
            torch.save(model.state_dict(), 'fasttext_cross_adamax_model.pt')
            best_model_score= eval_score

            

          

          




LR = 0.00050000, Hidden nodes = 1600

Epoch: 0 	Training Loss: -0.669022 	Validation Loss: -0.697163
Validation loss decreased (inf --> -0.697163).  Saving model ...
Epoch: 1 	Training Loss: -0.780866 	Validation Loss: -0.725529
Validation loss decreased (-0.697163 --> -0.725529).  Saving model ...
Epoch: 2 	Training Loss: -0.844695 	Validation Loss: -0.745111
Validation loss decreased (-0.725529 --> -0.745111).  Saving model ...
Epoch: 3 	Training Loss: -0.875266 	Validation Loss: -0.749840
Validation loss decreased (-0.745111 --> -0.749840).  Saving model ...
Epoch: 4 	Training Loss: -0.905009 	Validation Loss: -0.745165
Epoch: 5 	Training Loss: -0.904850 	Validation Loss: -0.745224
pred_shape (1379,)
result 0.7345
best_model_score increased (-inf --> 0.734500).  Saving model ...

LR = 0.00050000, Hidden nodes = 1800

Epoch: 0 	Training Loss: -0.617768 	Validation Loss: -0.708254
Validation loss decreased (inf --> -0.708254).  Saving model ...
Epoch: 1 	Training Loss: -0.792715 	Val

# Practice

In [ ]:
trainload= _load_data('sts-train.csv')

In [ ]:
data=_sample_pairs(trainload,2)

In [ ]:
data_len= []
for i in data['s0']:
  data_len.append(len(i))
max_len_sent=max(data_len)

rep= torch.zeros(max_len_sent, 2 ,300)
rep.shape

torch.Size([11, 2, 300])

In [ ]:
(data['s0'])

[['brown',
  'and',
  'white',
  'cow',
  'standing',
  'in',
  'grass',
  'at',
  'side',
  'of',
  'road'],
 ['boy', 'scouts', 'delay', 'decision', 'on', 'admitting', 'gays']]

In [ ]:
temp=[]
rep[0][0]=torch.tensor(indextovector[wordtoindex['brown']])


In [ ]:
rep[0]

tensor([[-0.3741, -0.0763,  0.1093,  0.1866,  0.0299,  0.1827, -0.6320,  0.1331,
         -0.1290,  0.6034, -0.6804, -0.1422, -0.1336, -0.6594,  0.0524,  0.1674,
          0.6392,  1.7680,  0.3462, -0.6248, -0.1287, -0.1970, -0.3745,  0.3306,
          0.0468, -0.6535, -0.5614,  0.2274,  0.2292, -0.4158, -0.1677,  0.3354,
          0.0972, -0.4670, -0.0269, -0.0677, -0.1921, -0.1337,  0.0163, -0.2083,
          0.6486, -0.1102, -0.0510,  0.0722,  0.1877,  0.2711, -0.3142,  0.1832,
          0.3915, -0.2255, -0.3819,  0.3306, -0.0899, -0.3763,  0.0480, -0.2050,
         -0.5489,  0.3044, -0.1887, -0.3034, -0.1157, -0.3487,  0.2800,  0.0501,
         -0.2814, -0.2335, -0.3683, -0.1204, -0.2833,  0.1857,  0.1036,  0.2531,
         -0.0340,  0.1051,  0.1214, -0.1630, -0.3318,  0.1731,  0.1076, -0.9936,
         -0.1171,  0.4223,  0.1512,  0.3106,  0.2674, -0.4927,  1.8049,  1.0738,
          0.3442,  0.1128, -0.1049,  0.3694,  0.4082, -0.4037,  0.3233, -0.0939,
         -0.0290,  0.3825, -

In [ ]:
data['m0'][0][11]

tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 

In [ ]:
m = nn.MaxPool1d(1)
input = torch.randn(6, 5)
input.shape

torch.Size([6, 5])

In [ ]:
input

tensor([[-0.7378,  1.7368,  0.0198,  0.8431,  0.9046],
        [-1.5884, -1.1110,  0.8694,  0.3053,  0.3550],
        [ 0.1539, -0.4941, -0.0704,  1.3168,  0.9116],
        [ 1.2474,  0.6942,  0.1928, -0.5697, -0.8765],
        [-0.5326, -0.9174,  0.2422, -0.1825,  0.2426],
        [-1.5224, -1.7681, -1.8564, -1.0241, -1.3065]])

In [ ]:
input.to(torch.float32)

tensor([[-0.7378,  1.7368,  0.0198,  0.8431,  0.9046],
        [-1.5884, -1.1110,  0.8694,  0.3053,  0.3550],
        [ 0.1539, -0.4941, -0.0704,  1.3168,  0.9116],
        [ 1.2474,  0.6942,  0.1928, -0.5697, -0.8765],
        [-0.5326, -0.9174,  0.2422, -0.1825,  0.2426],
        [-1.5224, -1.7681, -1.8564, -1.0241, -1.3065]])

In [ ]:
output = m(input)
output.shape

In [ ]:
out= torch.flatten(output, 1)

In [ ]:
out.shape

In [ ]:
a=torch.tensor(np.array([1,2,3]))
b=torch.tensor(np.array([4,5,6]))
a*b

In [ ]:
c=abs(a-b)

c

In [ ]:
import math

In [ ]:
torch.cat((a,b),1).shape

In [ ]:
c= torch.randn(2, 5)
d= torch.randn(2,5)
e= torch.cat((c,d),1)
e.shape


In [ ]:
(c-d)

In [ ]:
-1.5705-1.4329

In [ ]:
(c*d)

In [ ]:
-1.5705*1.4329

In [ ]:
-0.6561*-0.2841

In [ ]:
'''blocksize=2
k= ((li[block:block+blocksize], block) for block in range(0,len(li),blocksize))
for i,j in k:
    print(i,j)'''

In [ ]:
'''def worker(args):
        print(args[0])
        print(args[1])'''

In [ ]:
p= np.random.randn(1,5)
p

In [ ]:
plt.plot(p,'-*')

In [ ]:
p[1:5]

In [ ]:
d={'a':[1],'b':[2],'c':[3]}

In [ ]:
d1={}
d1= d.keys
d1

In [ ]:
d

In [ ]:
d2={}
d3=d2.fromkeys(d.keys(),[])

In [ ]:
d3

In [ ]:
d3['b'].append(909)

In [ ]:
d4 = copy.deepcopy(d3)
d4

In [ ]:
d4['a'].append(78)

In [ ]:
d4

In [ ]:
d['a'].append(909)

In [ ]:
d

In [ ]:
d5={}
for i in d.keys():
  d5[i]=[]

In [ ]:
d5

In [ ]:

d5['a'].append(78)

In [ ]:
d5